In [ ]:
from PIL import Image, ImageFilter
img = "perro.jpg"


with Image.open(img) as img_p:
    img_p.load()
img_p.show()
res = img_p.size
print("Resolución de imagen :", res)
img_perro = img_p.crop((75, 50, 660, 1200))  #recorte de imagen
img_perro.show()

In [ ]:
img_perro_gris = img_perro.convert("L")      #transformación a escala de grises
img_perro_gris.show()


In [ ]:
threshold = 140                        #umbral
#conversión de todos los pixeles con umbral mayor a 100 en blanco, resto negro.
#Se constata que no es buena la separación entre el perro y el fondo
img_perro_threshold = img_perro_gris.point(lambda x: 255 if x > threshold else 0)
img_perro_threshold.show()

In [ ]:
#Extracción de canales rojo, verde y azul (imagen original)
rojo, verde, azul = img_perro.split()
rojo.show()
verde.show()
azul.show()

In [ ]:
#El canal azul tiene un mayor contraste entre los píxeles que representan al perro y los que representan el fondo. 
#Utilizar la imagen del canal azul para establecer el umbral:
threshold = 140
img_perro_threshold_a = azul.point(lambda x: 255 if x > threshold else 0)
img_perro_threshold_a = img_perro_threshold_a.convert("1")
img_perro_threshold_a.show()

In [ ]:
#Definición de funciones para ciclos de 'erosión' y 'dilatación'
def erode(cycles, image):
    for _ in range(cycles):
        image = image.filter(ImageFilter.MinFilter(3))
    return image

def dilate(cycles, image):
    for _ in range(cycles):
        image = image.filter(ImageFilter.MaxFilter(3))
    return image

In [ ]:
#imagen de umbral erosionada - Eliminación del fondo de la imagen
#ciclos de erosión
paso_1 = erode(2, img_perro_threshold_a)
paso_1.show()

In [ ]:
#ciclos de dilatación
paso_2 = dilate(15, paso_1)
paso_2.show()

In [ ]:
#Nuevos ciclos de erosión - Finalización de la máscara
perro_mascara = erode(6, paso_2)
perro_mascara.show()

In [ ]:
#Eliminación de bordes afilados
#Conversión imagen binaria a una imagen en escala de grises
perro_mascara = perro_mascara.convert("L")
perro_mascara = perro_mascara.filter(ImageFilter.BoxBlur(8))
perro_mascara.show()

In [ ]:
#Extracción de la imagen del perro
blank = img_perro.point(lambda _: 0)
perro_recorte = Image.composite(img_perro, blank, perro_mascara)
perro_recorte.show()

In [ ]:
#Superposición de imágenes
filename_monastery = "habitacion.jpg"
with Image.open(filename_monastery) as img_habitacion:
    img_habitacion.load()

img_habitacion.paste(img_perro.resize((img_perro.width // 5, img_perro.height // 5)),(600, 600),
                     perro_mascara.resize((perro_mascara.width // 5, perro_mascara.height // 5)),)
img_habitacion.show()